In [1]:
#!pip install git+https://github.com/deepmipt/bert.git@feat/keras

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'1.13.1'

In [4]:
tf.enable_eager_execution()

In [5]:
tf.executing_eagerly()

True

In [6]:
from bert_dp import bert

In [7]:
bert_model = bert.BERT(name='bert')

In [8]:
inputs = {'pos_ids': tf.constant([[0, 1]]), 'segment_ids': tf.constant([[0, 0]]), 'token_ids': tf.constant([[1, 4]])}

In [9]:
mask = tf.constant([[1, 0]])

In [10]:
bert_model(inputs=inputs, mask=mask)

Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor: id=4165, shape=(1, 2, 768), dtype=float32, numpy=
array([[[-2.2176013 ,  1.0577515 ,  0.84728235, ..., -1.1046779 ,
          1.3490838 , -1.1778777 ],
        [-2.1938188 ,  1.071013  ,  0.88197637, ..., -1.1184927 ,
          1.3465396 , -1.1005738 ]]], dtype=float32)>

In [11]:
bert_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeddings (BERTCombinedEmbe multiple                  92206848  
_________________________________________________________________
embeddings/dropout (Dropout) multiple                  0         
_________________________________________________________________
embeddings/LayerNorm (LayerN multiple                  1536      
_________________________________________________________________
encoder (TransformerEncoder) multiple                  0         
_________________________________________________________________
pooler/dense (Dense)         multiple                  590592    
Total params: 92,798,976
Trainable params: 92,798,976
Non-trainable params: 0
_________________________________________________________________


In [12]:
tf.keras.utils.get_file(
    fname='bert',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/bert'

In [13]:
init_checkpoint = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
var_names = tf.train.list_variables(init_checkpoint)
check_point = tf.train.load_checkpoint(init_checkpoint)

In [14]:
checkpoint_vars = set()
for v, _ in var_names:
    checkpoint_vars.add(v)

In [15]:
checkpoint_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer_1',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 

In [16]:
len(checkpoint_vars)

209

In [17]:
model_vars = set()
for v in bert_model.variables:
    model_vars.add(v.name.split(':')[0])

In [17]:
model_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/pooler/dense/bias',
 'bert/pooler/dense/kernel'}

In [17]:
len(model_vars)

7

In [18]:
bert_model.variables

[<tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32, numpy=
 array([[-0.02514946,  0.01026926, -0.02847951, ...,  0.01424474,
         -0.03260387, -0.01046715],
        [ 0.00264451,  0.0291022 ,  0.01187141, ...,  0.00852287,
          0.00036667, -0.00320941]], dtype=float32)>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32, numpy=
 array([[-0.00683917,  0.01018448, -0.03188806, ..., -0.02704441,
          0.0003404 ,  0.01949455],
        [ 0.00346667,  0.0070805 ,  0.00665813, ...,  0.02054542,
         -0.003881  , -0.01862706],
        [-0.00774869, -0.01833956, -0.00218828, ...,  0.02550897,
         -0.01412613,  0.00711717],
        ...,
        [ 0.00097339,  0.00105322, -0.01415684, ..., -0.00699323,
          0.00946521, -0.0128039 ],
        [-0.00629447, -0.01220857,  0.03450704, ...,  0.00660803,
         -0.00150067,  0.01885889],
        [-0.01116214, -0.0172731 ,  0.02173424, ..., -0.01009039,
   

In [19]:
bert_model.load_weights(init_checkpoint)

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


AssertionError: Some objects had attributes which were not restored: {<bert_dp.embeddings.BERTCombinedEmbedding object at 0x7f5ab43489b0>: ['OBJECT_CONFIG_JSON'], <tensorflow.python.keras.layers.core.Dropout object at 0x7f5a880519e8>: ['OBJECT_CONFIG_JSON'], <bert_dp.normalization.LayerNormalization object at 0x7f5a88051fd0>: ['OBJECT_CONFIG_JSON'], <bert_dp.encoder.TransformerEncoder object at 0x7f5a88051f60>: ['OBJECT_CONFIG_JSON'], <tensorflow.python.keras.layers.core.Dense object at 0x7f5a807ed240>: ['OBJECT_CONFIG_JSON']}